<h1>Open TG-GATEs clinical pathology data from ChEMBL using web services</h1>

<h2>Introduction</h2>

Open TG-GATEs is a toxicogenomics database that stores gene expression profiles and traditional toxicological data derived from in vivo (rat) and in vitro (primary rat hepatocytes, primary human hepatocytes) exposure to 170 compounds at multiple dosages and time points. The toxicology data is composed of biochemistry, hematology and histopathology findings with pathology imaging from the in vivo studies and cytotoxicity from the in vitro studies. The 170 compounds include representative known liver- and kidney-injuring pharmaceuticals, compounds and chemicals.<sup>(1)</sup> The data can be browsed <a href='http://toxico.nibiohn.go.jp/open-tggates/english/search.html'>here</a>.

Biochemistry, hematology, organ weight and histopathology findings from the Open TG-GATEs dataset have been integrated into ChEMBL and can be found under src_id 11 and document_chembl_ids CHEMBL2766011 (biochemistry), CHEMBL2766014 (hematology), CHEMBL2766012 (organ weight) and CHEMBL2766013 (histopathology). Doses, time points, regimens and study_ids are stored in the assay description. For each biochemistry, hematology and organ weight endpoint, the average value across the individual animal samples has been calculated. The averaged treatment value is recorded as an activity standard_value while the  averaged controls for those endpoints can be found in the activity_comment.

Detailed information regarding how to use the ChEMBL web services can be found <a href='https://www.ebi.ac.uk/chembl/ws'>here</a>.

<sup>(1)</sup> Igarashi, Y., Nakatsu, N., Yamashita, T., Ono, A., Ohno, Y., Urushidani, T., & Yamada, H. (2015). Open TG-GATEs: a large-scale toxicogenomics database. Nucleic Acids Research, 43(Database issue), D921–7.



<h2>Configuration and setup</h2>

In [1]:
from chembl_webresource_client.new_client import new_client
from chembl_webresource_client.settings import Settings
Settings.Instance().MAX_LIMIT = 1000
import requests
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('max_colwidth',300)
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",150)

<h2> Retrieve Open TG-GATEs data from ChEMBL</h2>

Requests to ChEMBL can be made either by 

1\. Using the API provided by the Python package chembl_webresource_client:

In [2]:
activity = new_client.activity
activity.set_format('json')

tggates_act_api = activity.get(document_chembl_id=(['CHEMBL2766011', 'CHEMBL2766014', 'CHEMBL2766012', 'CHEMBL2766013']))

len(tggates_act_api)

158199

2\. Directly <i>via</i> URLs:

In [3]:
url_stem = "https://www.ebi.ac.uk"
tggates_url = requests.get(url_stem + "/chembl/api/data/activity.json?assay_description__icontains=TG-GATES&limit=1000").json()
tggates_act_url = tggates_url['activities']

while tggates_url['page_meta']['next']:
    tggates_url = requests.get(url_stem + tggates_url['page_meta']['next']).json()
    tggates_act_url += tggates_url['activities']

len(tggates_act_url)

158199

For simplicity, the retrieved data will be converted to a dataframe: 

In [4]:
tggates_act_api_list = list(tggates_act_api.all())

tggates_df = pd.DataFrame(tggates_act_api_list)

tggates_df.shape

(158199, 30)

As mentioned before, doses, time points, regimens and study_ids are stored in the assay description. For each biochemistry, hematology and organ weight endpoint, the average value across the individual animal samples has been calculated. The averaged treatment value is recorded as an activity standard_value while the averaged controls for those endpoints can be found in the activity_comment. Here is an example,

In [5]:
tggates_df.head(1)

,activity_comment,activity_id,assay_chembl_id,assay_description,assay_type,bao_endpoint,bao_format,canonical_smiles,data_validity_comment,document_chembl_id,document_journal,document_year,molecule_chembl_id,pchembl_value,potential_duplicate,published_relation,published_type,published_units,published_value,qudt_units,record_id,standard_flag,standard_relation,standard_type,standard_units,standard_value,target_chembl_id,target_organism,target_pref_name,uo_units
0,SD=34.06; p-value=0.76; N=5 || Control_MEAN=60.20; Control_SD=13.10; Control_N=5,13667396,CHEMBL2821586,Open TG-GATES: Regimen: Single; Time: 6 hr; Dose: Middle; Route: Gavage | Dataset: Biochemistry; Assay: TRIG (Triglycerides); Study_ID: 429/10,T,BAO_0000179,BAO_0000218,CN1C(=O)N(C)c2ncn(C)c2C1=O,None,CHEMBL2766011,None,None,CHEMBL113,None,False,None,TRIG,mg/dL,65.4,http://www.openphacts.org/units/MicrogramPerMilliliter,1933649,True,None,TRIG,ug.mL-1,654,CHEMBL376,Rattus norvegicus,Rattus norvegicus,UO_0000274


Open TG-GATEs data is now ready to be used.

Further processing of the data might be useful. In the next step, parameters such as dose_level, time, regimen, route, study_id were extracted from the assay_description. p-values and standard_values for the controls can also be extracted from the activity_comment for further comparison with the treatment standard_values.

In [6]:
# get fields of interest
tggates = tggates_df[['assay_chembl_id', 'assay_description', 'molecule_chembl_id', 'standard_type', 'standard_value', 'standard_units', 'activity_comment']]

tggates.shape

(158199, 7)

In [7]:
tggates.head(1)

,assay_chembl_id,assay_description,molecule_chembl_id,standard_type,standard_value,standard_units,activity_comment
0,CHEMBL2821586,Open TG-GATES: Regimen: Single; Time: 6 hr; Dose: Middle; Route: Gavage | Dataset: Biochemistry; Assay: TRIG (Triglycerides); Study_ID: 429/10,CHEMBL113,TRIG,654,ug.mL-1,SD=34.06; p-value=0.76; N=5 || Control_MEAN=60.20; Control_SD=13.10; Control_N=5


In [8]:
# filter on endpoints of interest, e.g. clinical chemistry and hematology data
h_cc = tggates[tggates['assay_description'].str.contains('Hematology', 'Biochemistry')]

h_cc.shape

(60540, 7)

In [10]:
# information regarding dose, time, regimen is store in assay_description
tggates_ap_split = h_cc['assay_description'].str.split(' \| |; ', 0, expand=True).add_prefix('assay_description_part')

tggates_ap_split.shape

(60540, 7)

In [11]:
tggates_ap_split.head(1)

,assay_description_part0,assay_description_part1,assay_description_part2,assay_description_part3,assay_description_part4,assay_description_part5,assay_description_part6
97659,Open TG-GATES: Regimen: Single,Time: 3 hr,Dose: Middle,Route: Gavage,Dataset: Hematology,Assay: MCHC (Ery. Mean Corpuscular HGB Concentration),Study_ID: 491/9


In [13]:
regimen    = tggates_ap_split['assay_description_part0'].str.split(':', 0,   expand=True).rename(columns={2:'regimen'}             ).drop([0,1], 1)
time       = tggates_ap_split['assay_description_part1'].str.split(':', 0,   expand=True).rename(columns={1:'time'}                ).drop([0],   1)
dose_level = tggates_ap_split['assay_description_part2'].str.split(':', 0,   expand=True).rename(columns={1:'dose_level'}          ).drop([0],   1)
route      = tggates_ap_split['assay_description_part3'].str.split(':', 0,   expand=True).rename(columns={1:'route'}               ).drop([0],   1)
dataset    = tggates_ap_split['assay_description_part4'].str.split(':', 0,   expand=True).rename(columns={1:'dataset'}             ).drop([0],   1)
assay_test = tggates_ap_split['assay_description_part5'].str.split(':', 0,   expand=True).rename(columns={1:'assay_test'}          ).drop([0],   1)
exp_group  = tggates_ap_split['assay_description_part6'].str.split(':|/', 0, expand=True).rename(columns={1:'exp_id', 2:'group_id'}).drop([0],   1)

In [14]:
regimen.head(1)

,regimen
97659,Single


In [15]:
exp_group.head(1)

,exp_id,group_id
97659,491,9


In [16]:
tggates_ap = (
    regimen
    .merge(time,       left_index=True, right_index=True, how='inner')
    .merge(dose_level, left_index=True, right_index=True, how='inner')
    .merge(route,      left_index=True, right_index=True, how='inner')
    .merge(dataset,    left_index=True, right_index=True, how='inner')
    .merge(assay_test, left_index=True, right_index=True, how='inner')
    .merge(exp_group,  left_index=True, right_index=True, how='inner')
)

In [17]:
tggates_ap.shape

(60540, 8)

In [18]:
tggates_ap.head(1)

,regimen,time,dose_level,route,dataset,assay_test,exp_id,group_id
97659,Single,3 hr,Middle,Gavage,Hematology,MCHC (Ery. Mean Corpuscular HGB Concentration),491,9


In [19]:
tggates_ap = tggates_ap.replace('^\s', '', regex=True)

In [20]:
# information regarding treatment and control is store in activity_comment:
tggates_split = h_cc['activity_comment'].str.split(' \|\| ', expand=True).add_prefix('act_comment_part')

tggates_split.shape

(60540, 2)

In [21]:
tggates_split.head(1)

,act_comment_part0,act_comment_part1
97659,SD=0.36; p-value=0.37; N=5,Control_MEAN=32.28; Control_SD=0.60; Control_N=5


In [22]:
tggates_treat   = tggates_split['act_comment_part0'].str.split(';', expand=True).add_prefix('treatment_part')
tggates_control = tggates_split['act_comment_part1'].str.split(';', expand=True).add_prefix('control_part')

tggates_treat.shape, tggates_control.shape

((60540, 3), (60540, 3))

In [23]:
tggates_treat.head(1)

,treatment_part0,treatment_part1,treatment_part2
97659,SD=0.36,p-value=0.37,N=5


In [24]:
t_sd_t = tggates_treat['treatment_part0'].str.split('=', expand=True).rename(columns={1:'SD'}).drop([0], 1)
t_sd_t.fillna(value=np.nan, inplace=True)
t_sd_t['SD'] = t_sd_t['SD'].map(lambda x: x.strip()).replace('nan', np.nan)

t_sd_t.shape

(60540, 1)

In [25]:
t_sd_t.head(1)

,SD
97659,0.36


In [26]:
t_p_t = tggates_treat['treatment_part1'].str.split('=', expand=True).rename(columns={1:'p-value'}).drop([0], 1)
t_p_t.fillna(value=np.nan, inplace=True)

t_p_t.shape

(60540, 1)

In [27]:
t_p_t.head(1)

,p-value
97659,0.37


In [28]:
t_n_t = tggates_treat['treatment_part2'].str.split('=', expand=True).rename(columns={1:'N'}).drop([0], 1)
t_n_t.fillna(value=np.nan, inplace=True)

t_n_t.shape

(60540, 1)

In [29]:
t_n_t.head(1)

,N
97659,5


In [30]:
tggates_control.head(1)

,control_part0,control_part1,control_part2
97659,Control_MEAN=32.28,Control_SD=0.60,Control_N=5


In [31]:
t_mean_c = tggates_control['control_part0'].str.split('=', expand=True).rename(columns={1:'control_value'}).drop([0], 1)
t_mean_c.fillna(value=np.nan, inplace=True)

t_mean_c.shape

(60540, 1)

In [32]:
t_mean_c.head(1)

,control_value
97659,32.28


In [33]:
t_sd_c = tggates_control['control_part1'].str.split('=', 0, expand=True).rename(columns={1:'control_SD'}).drop([0], 1)
t_sd_c.fillna(value=np.nan, inplace=True)

t_sd_c.shape

(60540, 1)

In [34]:
t_sd_c.head(1)

,control_SD
97659,0.60


In [36]:
t_n_c = tggates_control['control_part2'].str.split('=', 0, expand=True).rename(columns={1:'control_N'}).drop([0], 1)
t_n_c.fillna(value=np.nan, inplace=True)

t_n_c.shape

(60540, 1)

In [37]:
t_n_c.head(1)

,control_N
97659,5


In [38]:
tggates_comment = (
    t_sd_t[['SD']]
    .merge(t_p_t.loc[:,   ['p-value']],       left_index=True, right_index=True, how='inner')
    .merge(t_n_t.loc[:,   ['N']],             left_index=True, right_index=True, how='inner')
    .merge(t_mean_c.loc[:,['control_value']], left_index=True, right_index=True, how='inner')
    .merge(t_sd_c.loc[:,  ['control_SD']],    left_index=True, right_index=True, how='inner')
    .merge(t_n_c.loc[: ,  ['control_N']],     left_index=True, right_index=True, how='inner')
)

tggates_comment.shape

(60540, 6)

In [39]:
tggates_comment.head(1)

,SD,p-value,N,control_value,control_SD,control_N
97659,0.36,0.37,5,32.28,0.60,5


In [40]:
# merge the assay parameters, with the activity data for treatment and control assays
tggates_f = (
    tggates_ap
    .merge(h_cc,            left_index=True, right_index=True, how='inner')
    .merge(tggates_comment, left_index=True, right_index=True, how='inner')
)

tggates_f.shape

(60540, 21)

In [41]:
tggates_f.head(1)

,regimen,time,dose_level,route,dataset,assay_test,exp_id,group_id,assay_chembl_id,assay_description,molecule_chembl_id,standard_type,standard_value,standard_units,activity_comment,SD,p-value,N,control_value,control_SD,control_N
97659,Single,3 hr,Middle,Gavage,Hematology,MCHC (Ery. Mean Corpuscular HGB Concentration),491,9,CHEMBL2829599,Open TG-GATES: Regimen: Single; Time: 3 hr; Dose: Middle; Route: Gavage | Dataset: Hematology; Assay: MCHC (Ery. Mean Corpuscular HGB Concentration); Study_ID: 491/9,CHEMBL964,MCHC,32.58,%,SD=0.36; p-value=0.37; N=5 || Control_MEAN=32.28; Control_SD=0.60; Control_N=5,0.36,0.37,5,32.28,0.60,5


In [42]:
# reorder columns and select fields of interest
tggates_f = tggates_f[['molecule_chembl_id', 'dataset', 'assay_chembl_id', 'assay_test', 'regimen', 'dose_level', 'time', 'route', 'exp_id', 'group_id', 'standard_type', 'standard_value', 'standard_units', 'SD', 'p-value', 'N', 'control_value', 'control_SD', 'control_N']]

tggates_f.head(1)

,molecule_chembl_id,dataset,assay_chembl_id,assay_test,regimen,dose_level,time,route,exp_id,group_id,standard_type,standard_value,standard_units,SD,p-value,N,control_value,control_SD,control_N
97659,CHEMBL964,Hematology,CHEMBL2829599,MCHC (Ery. Mean Corpuscular HGB Concentration),Single,Middle,3 hr,Gavage,491,9,MCHC,32.58,%,0.36,0.37,5,32.28,0.60,5


<h1>Tox Flag</h1>

We are always interested in understanding how these datasets are used by our collaborators.
<br>For these type of data, we believe it would be useful to the user to have a simple flag that could summarise the observations across treatment and controls.</br>

Here we propose a way of providing this summary based on very simple and intuitive rules:

For each study and endpoint, a tox_flag is calculated based on comparisons between 

1\. the treatment values and the controls:
<br>- if there's a significant difference between treatment values and controls (p-value <= 0.05) and there's an increase above controls: tox_flag = 0.5</br>
<br>- if there's a significant difference between treatment values and controls (p-value <= 0.05) and there's a decrease below controls: tox_flag = -0.5</br>

2\. the treatment_values and normal ranges of historical controls
<br>- if there's a significant difference between treatment values and controls (p-value <= 0.05) and there's an increase above the normal range: tox_flag = 1</br>
<br>- if there's a significant difference between treatment values and controls (p-value <= 0.05) and there's a decrease below the normal range: tox_flag = -1</br>

Normal ranges available in the DrugMatrix<sup>2</sup> <i>in vivo</i> dataset were used as reference. The DrugMatrix dataset can also be found in CHEMBL under src_id 15 and document_chembl_ids CHEMBL2924216 (biochemistry), CHEMBL2924217 (hematology) and CHEMBL2924218 (pathology).

<sup>2</sup>Ganter, B., Tugendreich, S., Pearson, C. I., Ayanoglu, E., Baumhueter, S., Bostian, K. A., et al. (2005). Development of a large-scale chemogenomics database to improve drug candidate selection and to understand mechanisms of chemical toxicity and action. Journal of Biotechnology, 119(3), 219–244.

In [43]:
# define how to calculate tox_flag
def tox_flag(row):
    if (float(row['standard_value']) > float(row['control_value'])) & (float(row['standard_value']) > float(row['nrange_uvalue'])) & (float(row['p-value']) <= 0.05) & (pd.isnull(row['standard_value']) != True) & (pd.isnull(row['control_value']) != True) & (pd.isnull(row['p-value']) != True):
        val = 1
    elif (float(row['standard_value']) > float(row['control_value'])) & (float(row['standard_value']) <= float(row['nrange_uvalue'])) & (float(row['p-value']) <= 0.05) & (pd.isnull(row['standard_value']) != True) & (pd.isnull(row['control_value']) != True) & (pd.isnull(row['p-value']) != True):
        val = 0.5
    elif (float(row['standard_value']) > float(row['control_value'])) & (float(row['p-value']) > 0.05) & (pd.isnull(row['standard_value']) != True) & (pd.isnull(row['control_value']) != True) & (pd.isnull(row['p-value']) != True):
        val = 0
    elif (float(row['standard_value']) < float(row['control_value'])) & (float(row['p-value']) <= 0.05) & (float(row['standard_value']) < float(row['nrange_lvalue'])) & (pd.isnull(row['standard_value']) != True) & (pd.isnull(row['control_value']) != True) & (pd.isnull(row['p-value']) != True):
        val = -1
    elif (float(row['standard_value']) < float(row['control_value'])) & (float(row['p-value']) <= 0.05) & (float(row['standard_value']) >= float(row['nrange_lvalue'])) & (pd.isnull(row['standard_value']) != True) & (pd.isnull(row['control_value']) != True) & (pd.isnull(row['p-value']) != True):
        val = -0.5
    elif (float(row['standard_value']) < float(row['control_value'])) & (float(row['p-value']) > 0.05) & (pd.isnull(row['standard_value']) != True) & (pd.isnull(row['control_value']) != True) & (pd.isnull(row['p-value']) != True):
        val = 0
    elif (float(row['standard_value']) == float(row['control_value'])) & (pd.isnull(row['standard_value']) != True) & (pd.isnull(row['control_value']) != True):
        val = 0
    else:
        val = 0
    return val

In [45]:
# retrieve normal ranges from the drugmatrix data in ChEMBL for the biochemistry and hematology endpoints
drugmatrix_act_api      = activity.get(document_chembl_id=(['CHEMBL2924216', 'CHEMBL2924217']))
drugmatrix_act_api_list = list(drugmatrix_act_api.all())
drugmatrix_df           = pd.DataFrame(drugmatrix_act_api_list)
dmatrix                 = drugmatrix_df[['standard_type', 'activity_comment']]

# extract normal range from activity_comment
dmatrix_split  = dmatrix['activity_comment'].str.split('\ \|\|\ ', 0, expand=True).add_prefix('act_comment_part')
dmatrix_nrange = dmatrix_split['act_comment_part2'].str.split(':', 0, expand=True).add_prefix('nrange_part')
nrange         = dmatrix_nrange['nrange_part1'].str.split('-', 0, expand=True).rename(columns={0: 'nrange_lvalue',1:'nrange_uvalue'})

# for each endpoint, merge upper and lower values for the normal range
dmatrix.drop('activity_comment', axis=1, inplace=True)
dmatrix_nrange = dmatrix.merge(nrange, left_index = True, right_index = True, how='inner').drop_duplicates()

In [51]:
dmatrix_nrange.head(1)

,standard_type,nrange_lvalue,nrange_uvalue
0,CHLORIDE,97,107


In [52]:
# merge dmatrix_nrange to tggates_all
tggates_all = tggates_f.merge(dmatrix_nrange, on='standard_type', how='inner')

tggates_all.head(1)

,molecule_chembl_id,dataset,assay_chembl_id,assay_test,regimen,dose_level,time,route,exp_id,group_id,standard_type,standard_value,standard_units,SD,p-value,N,control_value,control_SD,control_N,nrange_lvalue,nrange_uvalue
0,CHEMBL964,Hematology,CHEMBL2829599,MCHC (Ery. Mean Corpuscular HGB Concentration),Single,Middle,3 hr,Gavage,491,9,MCHC,32.58,%,0.36,0.37,5,32.28,0.60,5,30.9,40.6


In [53]:
# calculate tox_flag
tggates_all['tox_flag'] = tggates_all.apply(tox_flag, axis=1)

tggates_all.head(1)

,molecule_chembl_id,dataset,assay_chembl_id,assay_test,regimen,dose_level,time,route,exp_id,group_id,standard_type,standard_value,standard_units,SD,p-value,N,control_value,control_SD,control_N,nrange_lvalue,nrange_uvalue,tox_flag
0,CHEMBL964,Hematology,CHEMBL2829599,MCHC (Ery. Mean Corpuscular HGB Concentration),Single,Middle,3 hr,Gavage,491,9,MCHC,32.58,%,0.36,0.37,5,32.28,0.60,5,30.9,40.6,0.0


In [59]:
# for clear visualisation, build matrix with rows of compounds and time points and columns of end poinst and dose_levels
tggates_all_pv = pd.pivot_table(tggates_all, index=['molecule_chembl_id','time'], columns=['standard_type', 'dose_level'], values='tox_flag')

tggates_all_pv.head(8)

standard_type             BASOLE             EOSLE              HCT       \
dose_level                  High  Low Middle  High  Low Middle High  Low   
molecule_chembl_id time                                                    
CHEMBL101          15 day    0.0  0.0    0.0   0.0  0.0    0.0 -0.5  0.0   
                   24 hr     0.0  0.0    0.0   0.0  0.0    0.0  0.0  0.0   
                   29 day    0.0  0.0    0.0   0.0  0.0    0.0 -0.5  0.0   
                   3 hr      0.0  0.0    0.0   0.0  0.0    0.0  0.0  0.0   
                   4 day     0.0  0.0    0.0   0.0  0.0    0.0  0.0  0.0   
                   6 hr      0.0  0.0    0.0   0.0  0.0    0.0  0.0  0.0   
                   8 day     0.0  0.0    0.0   0.0  0.0    0.0 -0.5  0.0   
                   9 hr      0.0  0.0    0.0   0.0  0.0    0.0  0.5  0.0   

standard_type                     HGB             LYMLE              MCH       \
dose_level                Middle High  Low Middle  High  Low Middle High  Low   
molecule_chembl_id time                                                         
CHEMBL101          15 day    0.0  1.0  0.0    0.0  -0.5  0.0    0.0  0.0  0.0   
                   24 hr     0.0  0.0  0.0    0.0  -1.0  0.0    0.0  0.0  0.0   
                   29 day    0.0  1.0  0.0    0.0   0.0  0.0    0.0  0.5  0.5   
                   3 hr      0.0  0.0  0.0    0.0   0.0  0.0    0.0  0.0  0.0   
                   4 day     0.0  0.0  0.0    0.0   0.0  0.0    0.0  0.0  0.0   
                   6 hr      0.0  0.0  0.0    0.0  -0.5 -0.5    0.0  0.0  0.0   
                   8 day     0.0  0.0  0.0    0.0  -0.5  0.0    0.0  0.0  0.0   
                   9 hr      0.5  1.0  0.0    1.0   0.0  0.0    0.0  0.0  0.0   

standard_type                    MCHC              MCV             MONOLE  \
dose_level                Middle High  Low Middle High  Low Middle   High   
molecule_chembl_id time                                                     
CHEMBL101          15 day    0.0  0.0  0.0    0.0 -0.5  0.0    0.0    0.0   
                   24 hr     0.0  0.0  0.0    0.0  0.0  0.0    0.0    0.0   
                   29 day    0.5  0.0  0.0    0.0  0.5  0.5    0.5    0.0   
                   3 hr      0.0 -0.5  0.0    0.0  0.0  0.0    0.0    0.0   
                   4 day     0.0  0.0  0.0    0.0  0.0  0.0    0.0    0.0   
                   6 hr      0.0  0.0  0.0    0.0  0.0  0.0    0.0    0.0   
                   8 day     0.0  0.0  0.0    0.5  0.0  0.0    0.0    0.0   
                   9 hr      0.0  0.0  0.0    0.0  0.0  0.0    0.0    0.0   

standard_type                         NEUTLE             PLAT              \
dose_level                 Low Middle   High  Low Middle High  Low Middle   
molecule_chembl_id time                                                     
CHEMBL101          15 day  0.0    0.0    0.5  0.0    0.0  1.0  1.0    0.0   
                   24 hr   0.0    0.0    0.5  0.0    0.0  0.0  0.0    0.0   
                   29 day  0.0   -0.5    0.0  0.0    0.0  1.0  0.0    0.0   
                   3 hr    1.0    0.0    0.0  0.0    0.0  0.0  0.0    0.0   
                   4 day   0.0    0.0    0.0  0.0    0.0  0.0  0.0    1.0   
                   6 hr    0.0    0.0    0.0  0.5    0.0  0.0  0.0    0.0   
                   8 day   0.0    0.0    0.5  0.0    0.0  0.0  0.0    0.0   
                   9 hr    0.0    0.0    0.0  0.0    0.0  0.0  0.0    0.0   

standard_type              RBC              WBC              
dose_level                High  Low Middle High  Low Middle  
molecule_chembl_id time                                      
CHEMBL101          15 day  0.0  0.0    0.0  0.0  0.0    0.0  
                   24 hr   0.0  0.0    0.0  0.0  0.0    0.0  
                   29 day  1.0  1.0    0.0  0.0  0.0    0.0  
                   3 hr    0.0  0.0    0.0  0.0  0.0    0.0  
                   4 day   0.0  0.0    0.0  0.0  0.0    0.0  
                   6 hr    0.0  0.0    0.0  0.0  0.0    0.0  
                 

Note that Open TG-GATEs gene expression profiles can be found in <a href='https://www.ebi.ac.uk/arrayexpress/search.html?query=Open+TG-GATEs'>ArrayExpress</a> and be linked to the Open TG-GATEs clinical pathology studies in ChEMBL.
<br>Taking as an example the experiment <a href='https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-800/?query=Open+TG-GATEs'>E-MTAB-800</a> from ArrayExpress,

1\. the 'Term Accession Number' in E-MTAB-800 corresponds to the molecule_chembl_id
<br>2\. the 'Source Name' in E-MTAB-800 has some correspondance to the exp_id and group_id. For example, the source name 182051 in E-MTAB-800 corresponds to exp_id = 182 and group_id = 5 (study_id = 182/5). The last digit in the source name refers to the animal_id.</br>
